In [1]:
import json
import requests
import os
import psycopg2
from datetime import date as d
import argparse
import csv
import itertools as it
from collections import OrderedDict
import pandas as pd
#import pandas_redshift
import sys
import yaml
import pandas.io.sql as sqlio

In [2]:
# local directory info -- !! Change these to your filepaths
data_science_repo_dir = '/Users/elsie/data-science-repo/'
creds_file='/Users/elsie/creds.yml'
# table info where long for diagnoses are
redshift_schema = 'data_science'
long_form_dx_table = 'bcop_dx_episode'
# redshift info for where to deposit processed data
redshift_table_name = 'data_science.elsie_episode_hcc_risk'
s3_bucket = 'clarify-datasource-bucket'
redshift_fpath = 'reference_data/df_redshift'
## default patient characteristics to use. WE can fix this to use real ones if we want to use actual HCC score
# default_patient_characteristics = {
#     'sex_cd' : 1,
#     'date_of_birth' : '1944-06-06',
#     'mcaid_cd' : 0
# }

In [3]:
sys.path.append(data_science_repo_dir)

conn_string = os.environ.get("DW_DEV_CONN")
conn = psycopg2.connect(conn_string)
cur = conn.cursor()
cur.execute("SET search_path TO data_science")
sql = """ SELECT DISTINCT episode_id, diagnosis_code, icd_version, '1' as sex_cd, 
'1944-06-06' as date_of_birth,  '0' as mcaid_cd FROM data_science.{}
WHERE episode_id IN ('09791223')""".format(long_form_dx_table)

episode_dx = sqlio.read_sql_query(sql, conn)
episode_dx.shape

(15, 6)

In [4]:
episode_dx.sort_values(by='episode_id')

,episode_id,diagnosis_code,icd_version,sex_cd,date_of_birth,mcaid_cd
0,09791223,I2510,0,1,1944-06-06,0
1,09791223,Z87891,0,1,1944-06-06,0
2,09791223,Z952,0,1,1944-06-06,0
3,09791223,F419,0,1,1944-06-06,0
4,09791223,T161XXA,0,1,1944-06-06,0
5,09791223,E785,0,1,1944-06-06,0
6,09791223,Z974,0,1,1944-06-06,0
7,09791223,C50919,0,1,1944-06-06,0
8,09791223,J029,0,1,1944-06-06,0
9,09791223,M1711,0,1,1944-06-06,0


In [5]:
episode_dx.sort_values(by=['episode_id'], inplace=True)
grouped = episode_dx.groupby('episode_id')
episode_group = [g for k, g in grouped]

In [6]:
episode_dx.groupby('episode_id')['diagnosis_code'].size()

episode_id
09791223    15
Name: diagnosis_code, dtype: int64

In [7]:
for k, g in episode_dx.groupby('episode_id')['diagnosis_code']:
    print (k)
    print (g)

09791223
0       I2510
1      Z87891
2        Z952
3        F419
4     T161XXA
5        E785
6        Z974
7      C50919
8        J029
9       M1711
10     Z01818
11        I10
12     M25561
13      G8918
14     Z01810
Name: diagnosis_code, dtype: object


### Send request to hcc web app

In [8]:
hcc_response = []
last_episode_id = ""
person_ind = 0
max_entries_in_query = 100

In [9]:
for i in range(0,len(episode_group)):
    print("Preparing query for group %s of %s" % (
        i + 1, len(episode_group)))

    if person_ind == 0:
        params = {
            "IDVAR": "EPISODE_ID",
            "KEEPVAR": "",
            "SEDITS": "1",
            "DATE_ASOF": d.today().isoformat(),
            "DF": "1"
        }
        params['persons'] = {}
        params['persons']['person'] = []

    this_query_result = episode_group[i]
    person = params['persons']['person']
    
    # Run through each person in the DB response to get info
    for ind, row in this_query_result.iterrows():
        if this_query_result.loc[ind]['episode_id'] != last_episode_id:
            last_episode_id = this_query_result.loc[ind]['episode_id']
            person.append({})
            this_person = person[person_ind]
            person_ind = person_ind + 1
            this_person['EPISODE_ID'] = this_query_result.loc[ind]['episode_id']
            this_person.update({
                    "SEX": str(this_query_result.loc[ind]['sex_cd']),
                    "DOB": this_query_result.loc[ind]['date_of_birth'],
                    "MCAID": str(this_query_result.loc[ind]['mcaid_cd']),
                    "NEMCAID": "0", # Assume not a NEW medicaid patient
                    "OREC": "0", # Assume entitlement due to old age, not disab
                    "DIAGS": {"DIAG":[]}
                })     
            diags_to_update = this_person['DIAGS']['DIAG']           
        diag_code = this_query_result.loc[ind]['diagnosis_code']
        if this_query_result.loc[ind]['icd_version'] == '9':
            diags_to_update.append({"DIAG":diag_code, "DIAG_TYPE":"9"})
        else:
            diags_to_update.append({"DIAG":diag_code, "DIAG_TYPE":"0"})
            
            
        ##################################################
        ## Submit our requests iteratively
        ##################################################
        
        if (person_ind < max_entries_in_query and i != len(episode_group) - 1) :
            continue
        
        #print("Submitting query for group %s of %s" % (i + 1, len(episode_group)))

        # Prepare headers
        url = "https://clarify-hcc-risk.herokuapp.com/api/V22/79/L1"

        headers = {
            'content-type': "application/json",
            'cache-control': "no-cache",
            'postman-token': "66655898-f05c-ac3a-7281-4597e777c744"
            }

        print ("params {}".format(params))
        # Submit request
        hcc_response.append(requests.post(url
                        , headers=headers
                        , data=json.dumps(params)))
        #print (hcc_response)
        person_ind = 0


Preparing query for group 1 of 1
params {'IDVAR': 'EPISODE_ID', 'KEEPVAR': '', 'SEDITS': '1', 'DATE_ASOF': '2019-03-17', 'DF': '1', 'persons': {'person': [{'EPISODE_ID': '09791223', 'SEX': '1', 'DOB': '1944-06-06', 'MCAID': '0', 'NEMCAID': '0', 'OREC': '0', 'DIAGS': {'DIAG': [{'DIAG': 'I2510', 'DIAG_TYPE': '0'}]}}]}}
params {'IDVAR': 'EPISODE_ID', 'KEEPVAR': '', 'SEDITS': '1', 'DATE_ASOF': '2019-03-17', 'DF': '1', 'persons': {'person': [{'EPISODE_ID': '09791223', 'SEX': '1', 'DOB': '1944-06-06', 'MCAID': '0', 'NEMCAID': '0', 'OREC': '0', 'DIAGS': {'DIAG': [{'DIAG': 'I2510', 'DIAG_TYPE': '0'}, {'DIAG': 'Z87891', 'DIAG_TYPE': '0'}]}}]}}
params {'IDVAR': 'EPISODE_ID', 'KEEPVAR': '', 'SEDITS': '1', 'DATE_ASOF': '2019-03-17', 'DF': '1', 'persons': {'person': [{'EPISODE_ID': '09791223', 'SEX': '1', 'DOB': '1944-06-06', 'MCAID': '0', 'NEMCAID': '0', 'OREC': '0', 'DIAGS': {'DIAG': [{'DIAG': 'I2510', 'DIAG_TYPE': '0'}, {'DIAG': 'Z87891', 'DIAG_TYPE': '0'}, {'DIAG': 'Z952', 'DIAG_TYPE': '0'}]}}]

In [10]:
######################################################
## collect response in cleaned up dataframe
######################################################

final_df = pd.concat([
    pd.DataFrame(json.loads(response.text)) for response in hcc_response
])
final_df.columns = map(lambda x: x.lower(), final_df.columns)
hcc_cols = list(filter(lambda x: x.startswith('hcc'), final_df.columns))
hcc_cols = sorted(hcc_cols, key=lambda x: int(x.split('hcc')[1]))
all_cols = ['episode_id'] + hcc_cols
subset_df = final_df[all_cols]
# how to do this without warning??
for col in hcc_cols:
    subset_df[col] = subset_df[col].astype(int)

######################################################
## write results back to redshift 
######################################################

# connect.connect_s3_redshift(
#     redshift_schema,
#     subset_df,
#     creds_file,
#     s3_bucket,
#     redshift_fpath)

# pandas_redshift.pandas_to_redshift(
#     data_frame = subset_df, 
#     redshift_table_name = redshift_table_name
#     )

/Users/elsie/miniconda3/envs/hcc/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [11]:
for e, g in subset_df.groupby('episode_id'):
    print (e)
    for c in g.columns:
        if (c != 'episode_id') & (int(g[c].sum()) > 0):
            print (" " + c)

09791223
